### Import required packages

In [1]:
import pandas as pd
import boto3
import json
import configparser
from botocore.exceptions import ClientError

### Fill out the redshift.cfg file. Before running the following cell

In [13]:
config = configparser.ConfigParser()
config.read_file(open('redshift.cfg'))

KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

DWH_CLUSTER_TYPE = config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE  = config.get('DWH','DWH_NODE_TYPE')

DWH_CLUSTER_IDENTIFIER = config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH','DWH_DB')
DWH_DB_USER = config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH','DWH_PORT')

DWH_IAM_ROLE = config.get('IAM_ROLE', 'NAME')
DWH_IAM_DESCRIPTION = config.get('IAM_ROLE', 'DESCRIPTION')
DWH_IAM_POLICY = config.get('IAM_ROLE', 'POLICY_ARN')

DWH_INBOUND_PROTOCOL = config.get('INBOUND_RULE', 'PROTOCOL')
DWH_INBOUND_CIDR = config.get('INBOUND_RULE', 'CIDR')

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", 'DWH_IAM_ROLE', 'DWH_IAM_DESCRIPTION', 'DWH_IAM_POLICY', 'DWH_INBOUND_PROTOCOL', 'DWH_INBOUND_CIDR'],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE, DWH_IAM_DESCRIPTION, DWH_IAM_POLICY, DWH_INBOUND_PROTOCOL, DWH_INBOUND_CIDR]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,sparkify
5,DWH_DB_USER,sparkifyuser
6,DWH_DB_PASSWORD,Musciman123!
7,DWH_PORT,5439
8,DWH_IAM_ROLE,dwhRole
9,DWH_IAM_DESCRIPTION,"""Allows RedShift clusters to call aws services."""


### Create clients

In [3]:
ec2 = boto3.resource('ec2', 
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

s3 = boto3.resource('s3',
                   region_name='us-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

iam = boto3.client('iam', 
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name='us-west-2')

redshift = boto3.client('redshift',
                         region_name='us-west-2',
                         aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET)

### Create IAM role

In [4]:
try:
    print('Creating IAM role...')
    dwhRole = iam.create_role(
                              Path='/',
                              RoleName=DWH_IAM_ROLE,
                              Description=DWH_IAM_DESCRIPTION,
                              AssumeRolePolicyDocument=json.dumps(
                              {
                                  'Statement': [
                                      {
                                          'Action': 'sts:AssumeRole',
                                          'Effect': 'Allow',
                                          'Principal': 
                                              {
                                                  'Service': 'redshift.amazonaws.com'
                                              }
                                      }
                                  ],
                                  'Version': '2012-10-17'
                              }))
except Exception as e:
    print(e)
    
print('Role Created...')
print('Attaching role...')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']
print('Role Attached...')
print('Getting role...')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE)['Role']['Arn']

print(roleArn)

Creating IAM role...
Role Created...
Attaching role...
Role Attached...
Getting role...
arn:aws:iam::049456562069:role/dwhRole


### Create the redshift cluster

In [5]:
try:
    response = redshift.create_cluster(
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        
        #Identifiers and Creds
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

### Query the status of the cluster

In [7]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,sparkifyuser
4,DBName,sparkify
5,Endpoint,"{'Address': 'dwhcluster.c9bowivkrlcx.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-981747e0
7,NumberOfNodes,4


### Get the endpoint value

In [18]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.c9bowivkrlcx.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::049456562069:role/dwhRole


### Open the TCP port

In [14]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0d4e622e')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


### Test connection to the cluster

In [16]:
%load_ext sql

try:
    conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
    print(conn_string)
    %sql $conn_string
    print('connected')
except Exception as e:
    print(e)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
connected


### Update the dwh.cfg with the values obtained here.

In [23]:
dwhUpdate = configparser.ConfigParser()

# update the cluster section
dwhUpdate.read('./dwh.cfg')
dwhUpdate.set('CLUSTER', 'host', DWH_ENDPOINT)
dwhUpdate.set('CLUSTER', 'db_name', DWH_DB)
dwhUpdate.set('CLUSTER', 'db_user', DWH_DB_USER)
dwhUpdate.set('CLUSTER', 'db_password', DWH_DB_PASSWORD)
dwhUpdate.set('CLUSTER', 'db_port', DWH_PORT)

#update the iam_role section
dwhUpdate.set('IAM_ROLE', 'arn', f'{DWH_ROLE_ARN}')

# write changes to dwh.cfg
with open('./dwh.cfg', 'w+') as config1:
    dwhUpdate.write(config1)

### Clean up cluster and delete role

In [24]:
# #### CAREFUL!!
# #-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
# #### CAREFUL!!

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

# #### CAREFUL!!
# #-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE)
# #### CAREFUL!!

{'ResponseMetadata': {'RequestId': '4d4ad22a-9043-4416-bac0-c28f7ae23ce7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4d4ad22a-9043-4416-bac0-c28f7ae23ce7',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Tue, 22 Dec 2020 20:52:42 GMT'},
  'RetryAttempts': 0}}